In [173]:
import numpy as np
import pandas as pd
import scipy
import os
def PR2dB(val):         # 2Do: add array support - REWRITE LATER
    return 10 * np.log10(val)

def VR2dB(val):         # 2Do: add array support - REWRITE LATER
    return 2 * PR2dB(val)

def GetAngle(alpha):
    return np.arctan2(np.sin(alpha), np.cos(alpha))

def set_beat(VSA: np.ndarray, VSB : np.ndarray, return_type = 'dB|deg') -> np.ndarray:
    ampl = np.sqrt(VSA[0]**2 + VSB[0]**2 + 2* VSA[0] * VSB[0] * np.cos(VSB[1] - VSA[1]))
    phase = np.arctan2((VSA[0] * np.sin(VSA[1]) + VSB[0] * np.sin(VSB[1])),(VSA[0] * np.cos(VSA[1]) + VSB[0] * np.cos(VSB[1])))
    type_ampl, type_phase = return_type.split('|')
    if type_ampl == 'dB':
        ampl = PR2dB(ampl)
    if type_phase == 'deg':
        phase = np.rad2deg(phase)
    return np.array([ampl,phase,])
def beat_function(S12,SRC,VAP):
    F = np.zeros(2)
    F[0] = -S12[0]**2 + SRC[0]**2 + VAP[0]**2 + 2 * SRC[0] * VAP[0] * np.cos(SRC[1] - VAP[1])
    F[1] = -np.tan(S12[1]) + (SRC[0]*np.sin(SRC[1]) + VAP[0]*np.sin(VAP[1]))/ (SRC[0]*np.cos(SRC[1]) + VAP[0] * np.cos(VAP[1]))
    return F
def read_data(file_dict : dict):
    absolute_path = os.path.join(file_dict['CD'],file_dict['file_name'])
    names = [['TimeStamp'],['Freq'],['Azimuth'],['Tilt']]
    element_MA = [['A{}_MAG'.format(i),'A{}_ARG'.format(i)] for i in range(4)]
    names.extend(element_MA)
    names = [item for element in names for item in element]
    data_frame = pd.read_csv(absolute_path,sep=';',comment = '#',header = None,names=names)
    return data_frame
def filter_data(df,freq,az,tilt,num_of_element,iloc):
    locls = locals()
    errors = 0
    for e in ['df', 'iloc', 'num_of_element']:
        locls.pop(e)
    for key in locls.keys():
        if locls[key] not in df.values:
            print(f'{key} = {locls[key]} not in DataFrame')
            errors = 1
    if errors == 1:
        return 0
    return df[((df.Freq == freq) & (df.Azimuth == az) & (df.Tilt == tilt))][['A{}_MAG'.format(num_of_element),'A{}_ARG'.format(num_of_element)]].iloc[iloc].to_numpy()
def convert_data(src,src_format, dst_format):
    if src_format == dst_format:
        return src
    else:
        src_mag_format, src_arg_format = src_format.split('|')
        dst_mag_format, dst_arg_format = dst_format.split('|')
        if src_mag_format != dst_mag_format:
            if src_mag_format == 'dB' and dst_mag_format == 'VR':
                src[0] = 10**(0.1*src[0])
            else:
                src[0] = PR2dB(src[0])
        if src_arg_format != dst_arg_format:
            if src_arg_format == 'deg' and dst_arg_format == 'rad':
                src[1] = np.deg2rad(src[1])
            else:
                src[1] = np.rad2deg(src[1])
        return src
def set_beat(VSA: np.ndarray, VSB : np.ndarray, return_type = 'dB|deg') -> np.ndarray:
    ampl = np.sqrt(VSA[0]**2 + VSB[0]**2 + 2* VSA[0] * VSB[0] * np.cos(VSB[1] - VSA[1]))
    phase = np.arctan2((VSA[0] * np.sin(VSA[1]) + VSB[0] * np.sin(VSB[1])),(VSA[0] * np.cos(VSA[1]) + VSB[0] * np.cos(VSB[1])))
    type_ampl, type_phase = return_type.split('|')
    if type_ampl == 'dB':
        ampl = PR2dB(ampl)
    if type_phase == 'deg':
        phase = np.rad2deg(phase)
    return np.array([ampl,phase,])
def beat_function(S12,SRC,VAP):
    F = np.zeros(2)
    F[0] = -S12[0]**2 + SRC[0]**2 + VAP[0]**2 + 2 * SRC[0] * VAP[0] * np.cos(SRC[1] - VAP[1])
    F[1] = -np.tan(S12[1]) + (SRC[0]*np.sin(SRC[1]) + VAP[0]*np.sin(VAP[1]))/ (SRC[0]*np.cos(SRC[1]) + VAP[0] * np.cos(VAP[1]))
    return F

# \\\ Macros section

In [303]:
df = read_data(dict(CD = 'C:\\Users\\vovan\\PycharmProjects\\GUI_Test\\Meas\\Meas_KTRV', file_name = 'Meas_27.05.23.txt'))
filtered_s12 = filter_data(df,freq = 1e9,az = 0,tilt = 0,num_of_element=3,iloc = 0)
filtered_src = filter_data(df,freq = 4e9,az = 0,tilt = 0,num_of_element=0,iloc = 9)
filtered_s12 = convert_data(filtered_s12,'dB|deg','VR|rad')
filtered_src = convert_data(filtered_src,'dB|deg','VR|rad')
fun = lambda VAP : beat_function(filtered_s12,filtered_src,VAP)
f0 = [filtered_s12[0],0]
ampl, phase  = scipy.optimize.fsolve(fun,f0,maxfev=1000)
interfer = np.array([ampl,GetAngle(phase + np.pi)])
result = set_beat(filtered_s12,interfer)
print(f'S12 ={convert_data(filtered_s12,"VR|rad","dB|deg")}')
print(f'SRC ={convert_data(filtered_src,"VR|rad","dB|deg")}')
print(f'Compensated  = {result}')


S12 =[-37.082174 -51.553852]
SRC =[-44.284054 147.574601]
Compensated  = [-44.284054 147.574601]


In [272]:
np.random.randn()

2.1164922762851344

In [110]:
deg = lambda val : np.deg2rad(val)
rad = lambda val : np.rad2deg(val)
VSA = [0.0005,deg(0)]
VSB = [0.0002,deg(170)]
VSR = set_beat(VSA,VSB,'VR|rad')
SRC = VSA
S12 = VSR
fun = lambda VAP : beat_function(S12,SRC,VAP)
ampl, phase  = scipy.optimize.fsolve(fun,[0,0])
result = np.array([ampl,GetAngle(phase + np.pi)])
phase = rad(GetAngle(phase))
print(f'Ampl = {ampl}, phase = {phase} deg')
set_beat(S12,result)


Ampl = 4.537253968548578e-08, phase = 5.089498156840467e-07 deg


array([-44.37385596, -38.4860302 ])